In [2]:
import pandas as pd
import datasets
import numpy as np
import os
import json

In [ ]:
file_to_visualize_path = "submissions/task1/retrieval_ranking/valid_combine_bm25_stella_ranker_minilml12.csv"
val_dataset_path = "Data/split/valid"
abstracts_corpus_path = "Data/climatecheck_abstracts.parquet"

In [4]:
abstracts_df = pd.read_parquet(abstracts_corpus_path)
val_dataset = datasets.load_from_disk(val_dataset_path)
file_to_visualize_df = pd.read_csv(file_to_visualize_path)


In [5]:
abstracts_df.head()

,title,abstract,abstract_id,merged_field_of_study
0,Exploring Spatiotemporal Relations between Soi...,An understanding of streamflow variability and...,0,[Environmental Science]
1,Annual net snow accumulation over southern Gre...,As part of NASA's Program for Arctic Regional ...,1,"[Environmental Science, Geology]"
2,Response of Westerly Jet Over the Northern Hem...,The westerly jet (WJ) in the upper troposphere...,2,"[Physics, Environmental Science, Geology]"
3,Collaborative Governance for Climate Change Ad...,Increasing climate change impacts are a major ...,3,"[Political Science, Environmental Science, Soc..."
4,Comparing Surface and Stratospheric Impacts of...,Geoengineering with stratospheric sulfate aero...,4,"[Environmental Science, Geology]"


In [6]:
val_dataset

Dataset({
    features: ['claim_id', 'abstract_id', 'claim', 'abstract', 'annotation'],
    num_rows: 227
})

In [7]:
file_to_visualize_df.head()

,claim_id,abstract_id,rank
0,29,175734,0
1,29,46997,1
2,29,327291,2
3,29,49783,3
4,29,365776,4


In [8]:
def visualize_sample(sample_id):
    sample = val_dataset[sample_id]
    sample_df = file_to_visualize_df[file_to_visualize_df['claim_id'] == sample['claim_id']]
    sample_df = sample_df.sort_values(by='rank', ascending=False)
    sample_df = sample_df.reset_index(drop=True)
    print(f"claim ID: {sample['claim_id']}")
    print(f"claim Text: \n{sample['claim']}\n\n")
    for i, row in sample_df.iterrows():
        abstract_id = row['abstract_id']
        abstract_text = abstracts_df[abstracts_df['abstract_id'] == abstract_id]['abstract'].values[0]
        gt_list = []
        for s in val_dataset:
            if s['claim_id'] == sample['claim_id']:
                gt_list.append(s['abstract_id'])
        print(f"Rank: {i+1}, {'correct' if abstract_id in gt_list else 'incorrect'}, \nAbstract ID: {abstract_id}, \nAbstract Text: \n{abstract_text}\n")
        print("---------------------------------------------------------------------")
        
    print("related abstracts in the data:")
    print("\n\n".join([abstracts_df[abstracts_df['abstract_id'] == abstract_id]['abstract'].values[0] for abstract_id in gt_list]))

In [15]:
visualize_sample(100)

claim ID: 570
claim Text: 
Wind and solar energy combined with battery storage are cleaner than fossil fuels


Rank: 1, incorrect, 
Abstract ID: 188495, 
Abstract Text: 
Make wind and solar power even cheaper by opening up access to the electricity grid and ending fossil-fuel subsidies, urge Gernot Wagner and colleagues.

---------------------------------------------------------------------
Rank: 2, incorrect, 
Abstract ID: 10717, 
Abstract Text: 
Climate change asks for the reduction in the consumption of fossil‐based fuels and an increased share of non‐regulated renewable energy sources, such as solar and wind power. In order to back up a larger share of these intermittent sources, ‘battery services’ are needed, currently provided only in large scale by hydropower, leading to more rapid and frequent changes in flows (hydropeaking) in the downstream rivers. Increased knowledge about the ecosystem response to such operations and design of cost‐effective measures is needed.

-----------

In [14]:
visualize_sample(80)

claim ID: 510
claim Text: 
Turns out, the rising temps in the Arctic during winter aren't due to heat coming from the ocean. 🤔


Rank: 1, incorrect, 
Abstract ID: 130145, 
Abstract Text: 
During the past three decades, the most rapid warming at the surface has occurred during the Arctic winter. By analyzing daily ERA-Interim data, it is found that the majority of the winter warming trend north of 70°N can be explained by the trend in the downward infrared radiation (IR). This downward IR trend can be attributed to an enhanced poleward flux of moisture and sensible heat into the Arctic by poleward-propagating Rossby waves, which increases the total column water and temperature within this region. This enhanced moisture flux is mostly due to changes in the planetary-scale atmospheric circulation rather than an increase in moisture in lower latitudes. The results of this study lead to the question of whether Arctic amplification has mostly arisen through changes in the Rossby wave respons